## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "default"
study_type = "program"
mode_of_interest = "train"
include_test_users = False
dynamic_labels = { }
use_imperial = True

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
# Do not run this notebook at all unless it is for a program; nbclient will run up through this cell
if study_type != "program":
    ipython = get_ipython()
    ipython._showtraceback = scaffolding.no_traceback_handler
    raise Exception("The plots in this notebook are only relevant to programs")

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_re
%store -r dic_pur

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dynamic_labels,
                                                                            dic_re,
                                                                            dic_pur=dic_pur,
                                                                            include_test_users=include_test_users)

In [ ]:
if 'mode_confirm' in expanded_ct.columns:
    mode_of_interest_df = expanded_ct.query(f"mode_confirm == '{mode_of_interest}'")
    debug_df.loc[f"{mode_of_interest}_trips"] = len(mode_of_interest_df)
    debug_df.loc[f"{mode_of_interest}_trips_with_replaced_mode"] = scaffolding.trip_label_count("Replaced_mode", mode_of_interest_df)

## Metrics for Specific Mode

In [ ]:
data_eb = expanded_ct.query(f"mode_confirm == '{mode_of_interest}'") if "mode_confirm" in expanded_ct.columns else expanded_ct

In [ ]:
quality_text = scaffolding.get_quality_text(expanded_ct, data_eb, mode_of_interest, include_test_users)

### Trips by purpose for specified mode

In [ ]:
plot_title_no_quality=f"Number of trips for each purpose for {mode_of_interest} only"
file_name= f'ntrips_{mode_of_interest}_purpose%s' % file_suffix

try:
    labels_tp = data_eb['Trip_purpose'].value_counts(dropna=True).keys().tolist()
    values_tp = data_eb['Trip_purpose'].value_counts(dropna=True).tolist()
    plot_title= plot_title_no_quality+"\n"+quality_text
    pie_chart_purpose(plot_title,labels_tp,values_tp,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_tp, labels_tp), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

In [ ]:
plot_title_no_quality=f"Number of trips for each replaced transport mode for {mode_of_interest} only"
file_name =f'ntrips_{mode_of_interest}_replaced_mode%s' % file_suffix

try:
    labels_eb = data_eb.Replaced_mode.value_counts(dropna=True).keys().tolist()
    values_eb = data_eb.Replaced_mode.value_counts(dropna=True).tolist()
    plot_title= plot_title_no_quality+"\n"+quality_text
    pie_chart_mode(plot_title,labels_eb,values_eb,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_eb, labels_eb), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Miles for each mode replaced by the specified mode

In [ ]:
if use_imperial:
    plot_title_no_quality=f"Distribution of Miles Replaced by {mode_of_interest}"
else:
    plot_title_no_quality=f"Distribution of Kilometers Replaced by {mode_of_interest}"
file_name =f'miles_{mode_of_interest}_replaced_mode%s' % file_suffix

try:
    if use_imperial:
        dg=data_eb.groupby('Replaced_mode').agg({'distance_miles': ['sum', 'count' , 'mean']},)
        dg.columns = ['Total (miles)', 'Count' ,'Average (miles)']
        dg = dg.reset_index()
        dg = dg.sort_values(by=['Total (miles)'], ascending=False)

        dg_dict = dict(zip(dg['Replaced_mode'], dg['Total (miles)']))
    else:
        dg=data_eb.groupby('Replaced_mode').agg({'distance_kms': ['sum', 'count' , 'mean']},)
        dg.columns = ['Total (kilometers)', 'Count' ,'Average (kilometers)']
        dg = dg.reset_index()
        dg = dg.sort_values(by=['Total (kilometers)'], ascending=False)

        dg_dict = dict(zip(dg['Replaced_mode'], dg['Total (kilometers)']))
    
    labels_m = []
    values_m = []

    for x, y in dg_dict.items():
        labels_m.append(x)
        values_m.append(y)

    plot_title= plot_title_no_quality+"\n"+quality_text
    pie_chart_mode(plot_title,labels_m,values_m,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_m, labels_m), file_name, plot_title)
    print(dg)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Average miles per trip for specified mode

In [ ]:
if use_imperial:
    plot_title_no_quality="Average Miles for each replaced mode with > 3 entries\n'Other' represents trips with a non-standard or missing replacement"
else:
    plot_title_no_quality="Average Kilometers for each replaced mode with > 3 entries\n'Other' represents trips with a non-standard or missing replacement"

file_name ='average_miles_replaced_mode%s' % file_suffix

try:
    if use_imperial:
        data = dg.drop((dg.query("Count < 3").index)).sort_values(by=['Average (miles)'], ascending=False)
        x='Replaced_mode'
        y='Average (miles)'
        y2 = "Count"

        plot_title= plot_title_no_quality+"\n"+quality_text
        barplot_mode(data,x,y,plot_title, expanded_ct['Replaced_mode'].dropna().unique().tolist(), file_name)
        alt_text = store_alt_text_bar(pd.DataFrame(data['Average (miles)'].values, data.Replaced_mode), file_name, plot_title)
    else:
        data = dg.drop((dg.query("Count < 3").index)).sort_values(by=['Average (kilometers)'], ascending=False)
        x='Replaced_mode'
        y='Average (kilometers)'
        y2 = "Count"

        plot_title= plot_title_no_quality+"\n"+quality_text
        barplot_mode(data,x,y,plot_title, expanded_ct['Replaced_mode'].dropna().unique().tolist(), file_name)
        alt_text = store_alt_text_bar(pd.DataFrame(data['Average (kilometers)'].values, data.Replaced_mode), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Number of trips by day for specified mode

In [ ]:
plot_title_no_quality=f"Number of {mode_of_interest} trips by day"
file_name =f'ntrips_{mode_of_interest}_per_day%s' % file_suffix


try:
    fq_days = data_eb.groupby(['start_local_dt_day']).agg({'start_local_dt_day': ['sum', 'count']})
    fq_days = fq_days.reset_index()
    fq_days.columns = ['Day of the Month', 'Total', 'Number of Trips']

    data = fq_days
    x = 'Day of the Month'
    y = 'Number of Trips'

    plot_title= plot_title_no_quality+"\n"+quality_text
    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Day of the Month'].values), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Number of trips by day of week¶

In [ ]:
plot_title_no_quality=f"Number of {mode_of_interest} trips by weekday"
file_name =f'ntrips_{mode_of_interest}_per_weekday%s' % file_suffix

try:
    fq_weekdays = data_eb.groupby(['start_local_dt_weekday']).agg({'start_local_dt_weekday': ['sum', 'count']})
    fq_weekdays = fq_weekdays.reset_index()
    fq_weekdays.columns = ['Weekday', 'Total', 'Number of Trips']
    weekday_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    fq_weekdays["Weekday"] = fq_weekdays.Weekday.apply(lambda x: weekday_labels[x])

    data = fq_weekdays
    x = 'Weekday'
    y = 'Number of Trips'

    plot_title= plot_title_no_quality+"\n"+quality_text
    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Weekday'].values), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)